# Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.
## Just make sure:
1- to add enough Markdown cells to explain what you decided to do and to report any observations you make.

2-to generate maps to visualize your neighborhoods and how they cluster together.

In [1]:


# importing old libraries (also used in the two first parts)
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup as bs



In [4]:
# importing new libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [8]:
df_toronto = pd.read_csv('Desktop/toronto_base.csv')
df_toronto.head(7)

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
5,5,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029
6,6,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577


## Create map for toronto

In [56]:
# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6532; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto.save("my_map1.html" )

In [57]:
from IPython.display import HTML

HTML('<iframe src=my_map1.html width=700 height=450></iframe>')

In [12]:
# @hiddel_cell
CLIENT_ID = 'FPIENWUBAFUUMXHPL5QK0LZICLXB1YAPUAHHPQCPVU3CUJB2' # your Foursquare ID
CLIENT_SECRET = 'CJBSSJYWDAUKOG40C5CFOJVYVILMAHI2AYPNBZGUWDX0Y1VJ' # your Foursquare Secret
VERSION = '20190530' # Foursquare API version

In [13]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(7)

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
5,5,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029
6,6,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577


## Create a map of Scarborough and its neighbourhoods

In [14]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude_scar, longitude_scar))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [15]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb.save('my_map2.html')


In [52]:
from IPython.display import HTML

HTML('<iframe src=my_map2.html width=700 height=450></iframe>')

## Get the top 100 venues in the neighborhood 'Steeles West', from Scarborough

In [16]:


neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))



Latitude and longitude values of "Rouge, Malvern" are 43.8066863, -79.19435340000003.


In [17]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [18]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cc951a91ed2196c4218df62'},
 'response': {'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.775537,
          'lng': -79.256833}],
        'distance

In [19]:


def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']



In [20]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
3,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
4,Chipotle Mexican Grill,Mexican Restaurant,43.776410,-79.258069
5,Scarborough Town Centre,Shopping Mall,43.775237,-79.257363
6,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645
7,Coliseum Scarborough Cinemas,Movie Theater,43.775995,-79.255649
8,Jimmy The Greek,Greek Restaurant,43.775112,-79.257119
9,Hot Topic,Clothing Store,43.775450,-79.257929


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

44 venues were returned by Foursquare.


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Get venues for each neighborhood in Scarborough

In [23]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Morningside, West Hill
Woburn
Scarborough Village
Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside
Birch Cliff
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Tam O'Shanter
Agincourt North, Milliken
Upper Rouge


In [24]:
scarborough_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa
5,"Morningside, West Hill",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
6,"Morningside, West Hill",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
7,"Morningside, West Hill",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center
8,"Morningside, West Hill",43.763573,-79.188711,Lawrence Ave E & Kingston Rd,43.767704,-79.189490,Intersection
9,"Morningside, West Hill",43.763573,-79.188711,Eggsmart,43.767800,-79.190466,Breakfast Spot


In [25]:
scarborough_venues.tail(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
55,Tam O'Shanter,43.781638,-79.304302,Kub Khao,43.780438,-79.299837,Thai Restaurant
56,Tam O'Shanter,43.781638,-79.304302,Little Caesars,43.780563,-79.298624,Pizza Place
57,Tam O'Shanter,43.781638,-79.304302,KFC,43.779440,-79.303371,Fast Food Restaurant
58,Tam O'Shanter,43.781638,-79.304302,Popeyes Louisiana Kitchen,43.780476,-79.298460,Fried Chicken Joint
59,Tam O'Shanter,43.781638,-79.304302,Enterprise Rent-A-Car,43.779370,-79.306530,Rental Car Location
60,Tam O'Shanter,43.781638,-79.304302,Gusto Pizza,43.783607,-79.298983,Pizza Place
61,Tam O'Shanter,43.781638,-79.304302,Rexall,43.780732,-79.298257,Pharmacy
62,"Agincourt North, Milliken",43.815252,-79.284577,Lickety's,43.816136,-79.287944,Coffee Shop
63,"Agincourt North, Milliken",43.815252,-79.284577,Port Royal Park,43.815477,-79.289773,Park
64,"Agincourt North, Milliken",43.815252,-79.284577,Milliken Public School Playground,43.815383,-79.289867,Playground


In [26]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, Milliken",3,3,3,3,3,3
Birch Cliff,4,4,4,4,4,4
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Cliffcrest, Cliffside",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",6,6,6,6,6,6
"Highland Creek, Rouge Hill, Port Union",1,1,1,1,1,1
"Ionview, Kennedy Park",4,4,4,4,4,4
"Maryvale, Wexford",7,7,7,7,7,7


In [27]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 45 uniques categories.


In [28]:
# one hot encoding
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]

scarb_onehot.head()

,Neighborhood,American Restaurant,Auto Garage,Bakery,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,"Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [29]:
scarb_onehot.shape

(65, 46)

In [30]:
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()
scarb_grouped.head(7)

,Neighborhood,American Restaurant,Auto Garage,Bakery,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Chinese Restaurant,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.0,0.000000,0.0,0.25,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.25,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.25,0.0,0.25,0.0,0.000000,0.0,0.0,0.000000
1,"Agincourt North, Milliken",0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.00,0.000000,0.333333,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.333333,0.000000,0.0,0.0,0.333333,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000
2,Birch Cliff,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.25,0.000000,0.000000,0.25,0.0,0.0,0.0,0.0,0.000000,0.0,0.25,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.25,0.0,0.000000,0.0,0.0,0.000000
3,"Clairlea, Golden Mile, Oakridge",0.0,0.0,0.222222,0.0,0.00,0.222222,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.111111,0.0,0.00,0.000000,0.0,0.111111,0.0,0.0,0.000000,0.00,0.0,0.111111,0.0,0.0,0.0,0.0,0.111111,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.111111,0.0,0.0,0.000000
4,"Cliffcrest, Cliffside",0.5,0.0,0.000000,0.0,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.5,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000
5,"Dorset Park, Scarborough Town Centre, Wexford ...",0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.00,0.166667,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.333333,0.0,0.000000,0.0,0.0,0.166667,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.166667,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.166667
6,"Highland Creek, Rouge Hill, Port Union",0.0,0.0,0.000000,1.0,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.000000


## Get top 10 venues per neighborhood

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Breakfast Spot,Sandwich Place,Vietnamese Restaurant,Convenience Store,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
1,"Agincourt North, Milliken",Coffee Shop,Playground,Park,College Stadium,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,Birch Cliff,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,Fried Chicken Joint,Fast Food Restaurant
3,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Soccer Field,Fast Food Restaurant,Metro Station,Intersection,Park,Vietnamese Restaurant,Department Store,Indian Restaurant
4,"Cliffcrest, Cliffside",American Restaurant,Motel,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
5,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Vietnamese Restaurant,Pet Store,Latin American Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
6,"Highland Creek, Rouge Hill, Port Union",Bar,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
7,"Ionview, Kennedy Park",Bus Station,Discount Store,Coffee Shop,Department Store,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint
8,"Maryvale, Wexford",Auto Garage,Bakery,Smoke Shop,Shopping Mall,Sandwich Place,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Convenience Store,General Entertainment
9,"Morningside, West Hill",Intersection,Spa,Breakfast Spot,Electronics Store,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,Convenience Store,General Entertainment


## Run k-means to cluster the neighborhoods into 5 clusters

In [38]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(14)
# set number of clusters
kclusters = 5

scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
#len(kmeans.labels_)#=16
#scarborough_data.shape

array([1, 0, 1, 1, 4, 1, 2, 1, 1, 1])

## Include kmeans.labels_ into the original Scarborough dataframe

In [39]:
scarb_merged = scarb_data

# add clustering labels
scarb_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

scarb_merged

,Unnamed: 0,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,Italian Restaurant,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0,Bar,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,2,M1E,Scarborough,"Morningside, West Hill",43.763573,-79.188711,1,Intersection,Spa,Breakfast Spot,Electronics Store,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,Convenience Store,General Entertainment
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Insurance Office,Convenience Store,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,4,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Playground,Convenience Store,Vietnamese Restaurant,College Stadium,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,5,M1K,Scarborough,"Ionview, Kennedy Park",43.727929,-79.262029,1,Bus Station,Discount Store,Coffee Shop,Department Store,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint
6,6,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,2,Bakery,Bus Line,Soccer Field,Fast Food Restaurant,Metro Station,Intersection,Park,Vietnamese Restaurant,Department Store,Indian Restaurant
7,7,M1M,Scarborough,"Cliffcrest, Cliffside",43.716316,-79.239476,1,American Restaurant,Motel,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
8,8,M1N,Scarborough,Birch Cliff,43.692657,-79.264848,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,Fried Chicken Joint,Fast Food Restaurant
9,9,M1P,Scarborough,"Dorset Park, Scarborough Town Centre, Wexford ...",43.757410,-79.273304,1,Indian Restaurant,Vietnamese Restaurant,Pet Store,Latin American Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


## Visualize the clusters in the map

In [43]:


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters.save('my_map3.html')


In [51]:
from IPython.display import HTML

HTML('<iframe src=my_map3.html width=700 height=450></iframe>')

C:\Users\HP\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Examine each of the five clusters

In [46]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,-79.160497,0,Bar,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
11,M1S,-79.262029,0,Lounge,Skating Rink,Breakfast Spot,Sandwich Place,Vietnamese Restaurant,Convenience Store,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
13,M1V,-79.284577,0,Coffee Shop,Playground,Park,College Stadium,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [47]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,Italian Restaurant,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
2,M1E,-79.188711,1,Intersection,Spa,Breakfast Spot,Electronics Store,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,Convenience Store,General Entertainment
3,M1G,-79.216917,1,Coffee Shop,Korean Restaurant,Insurance Office,Convenience Store,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
5,M1K,-79.262029,1,Bus Station,Discount Store,Coffee Shop,Department Store,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint
7,M1M,-79.239476,1,American Restaurant,Motel,Convenience Store,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
8,M1N,-79.264848,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Convenience Store,Insurance Office,Indian Restaurant,Fried Chicken Joint,Fast Food Restaurant
9,M1P,-79.273304,1,Indian Restaurant,Vietnamese Restaurant,Pet Store,Latin American Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
12,M1T,-79.304302,1,Pizza Place,Chinese Restaurant,Noodle House,Italian Restaurant,Thai Restaurant,Fried Chicken Joint,Fast Food Restaurant,Rental Car Location,Pharmacy,Electronics Store


In [48]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1L,-79.284577,2,Bakery,Bus Line,Soccer Field,Fast Food Restaurant,Metro Station,Intersection,Park,Vietnamese Restaurant,Department Store,Indian Restaurant


In [49]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M1R,-79.295849,3,Auto Garage,Bakery,Smoke Shop,Shopping Mall,Sandwich Place,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Convenience Store,General Entertainment


In [50]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M1J,-79.239476,4,Playground,Convenience Store,Vietnamese Restaurant,College Stadium,Insurance Office,Indian Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
